## **1. 군집화**

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('review_usa.csv')

In [3]:
print(data.shape)
print(data.columns)

(12572, 12)
Index(['ReviewerID', 'Review', 'splited_sentence', 'cleaned_sentence',
       'detect_language', 'sentiment_result', 'negative', 'neutral',
       'positive', 'Review Score', 'Review date', 'Review Like'],
      dtype='object')


In [4]:
data['cleaned_sentence']
df = data['cleaned_sentence']

In [5]:
import re
import nltk
from sentence_transformers import SentenceTransformer, util  #util 모듈은 문장 임베딩 생성, 비교, 유사도 계산 등의 작업

c:\python\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
## 일단 이름이 너무 이상해서바꾸는 함수 만들기
import pandas as pd
import re
import emoji

# 대체 패턴 정리하기
deniro_patterns = [
    r'robot de Niro',r'De nitro',r'DeNiro',r'robot De nitro',r'robot de nitro',
    r'robot desire',r'robot d nitro',r'robot De hero',r'robot De tiro',r'robot',r'robot desirous', r"robot's", r'robot DiNero',r'robot D',
    r'robot i nitro', r'robot desert', r'robot danio', r'robot Deniro',r'robots', r'robot D Niro',r'robot DiNero',r"Robert D'Niro",r"De tiro's", r'Robert desirous', r'Mr. desire',r'Di nitro'
]  

hathaway_patterns = [
    r'hatchway\'s', r'anna pathway', r'pathway', r'acne\'s', r'Ann', r'anna Hathaway', r'anna Hathaways\'s', r'anna hatchway', r'anna gateway', r'anna halfway', r'anna waterway']

Jules_patterns = [r'rules'] 

Nancy_meyers_patterns = [r'Nancy meter\'s', r'Nancy meters', r'Nancy meter']

Oscar_patterns = [r'scar winner']

Two_patterns = [r'robot and anna', r'anna & robot\'s']

# 정규 표현식
deniro_regex = re.compile('|'.join(deniro_patterns), re.IGNORECASE)
hathaway_regex = re.compile('|'.join(hathaway_patterns), re.IGNORECASE)
jules_regex = re.compile('|'.join(Jules_patterns), re.IGNORECASE)
nancy_regex = re.compile('|'.join(Nancy_meyers_patterns), re.IGNORECASE)
two_regex = re.compile('|'.join(Two_patterns), re.IGNORECASE)
oscar_regex = re.compile('|'.join(Oscar_patterns), re.IGNORECASE)

#바꾸기!!
def replace_patterns(text):
    if isinstance(text, str):
        text = deniro_regex.sub('Deniro', text)
        text = hathaway_regex.sub('Hathaway', text)
        text = jules_regex.sub('Jules', text)
        text = nancy_regex.sub('Nancy Meyers', text)
        text = oscar_regex.sub('Oscar', text)
        text = two_regex.sub('Deniro and Hathaway', text)
    return text

In [7]:
df = df.apply(replace_patterns)

In [8]:
df.head()

0                              Wasn't what I expected.
1                                   Really enjoyed it.
2        Including the hope for redemption at the end.
3    A true feel good movie that I wish I saw in th...
4    Hathaway and Deniro have great chemistry and t...
Name: cleaned_sentence, dtype: object

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
corpus = df.values.tolist()
corpus_embeddings = embedder.encode(corpus)


c:\python\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
from sklearn.cluster import KMeans
num_clusters = 20
random_state = 2024
clustering_model = KMeans(n_clusters=num_clusters,random_state=random_state)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_  #0~19까지 정수 값 포함

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

cluster_dict = {}
for i, cluster in enumerate(clustered_sentences):
    cluster_dict[f"review_cluster_{i}"] = cluster

In [11]:
for k, v in cluster_dict.items():
    print(k, len(v))

review_cluster_0 407
review_cluster_1 1157
review_cluster_2 436
review_cluster_3 476
review_cluster_4 895
review_cluster_5 614
review_cluster_6 331
review_cluster_7 1217
review_cluster_8 535
review_cluster_9 504
review_cluster_10 289
review_cluster_11 601
review_cluster_12 492
review_cluster_13 1005
review_cluster_14 714
review_cluster_15 630
review_cluster_16 566
review_cluster_17 427
review_cluster_18 629
review_cluster_19 647


In [12]:
print(len(cluster_dict.keys()),len(cluster_dict.values()))

20 20


In [13]:
na_cluster_df = pd.DataFrame(
    {"cluster": cluster_dict.keys(), "sentences": cluster_dict.values()}
    )

cluster_df = pd.DataFrame()

for k,v in cluster_dict.items():
    df = pd.DataFrame({"review": v})
    df['cluster'] = k
    cluster_df = pd.concat([cluster_df, df])

In [14]:
cluster_df
cluster_df = cluster_df.drop_duplicates(subset=['review'])

In [15]:
cluster_df.to_excel('전처리_북미_kmeans_hugging.xlsx')

PermissionError: [Errno 13] Permission denied: '전처리_북미_kmeans_hugging.xlsx'

위 오류: 이미 있어서..

## **2. 요약**

In [55]:
from prettyprinter import pprint
from transformers import pipeline
import textwrap

In [56]:
cluster_df

,review,cluster
0,A lot of shaming the new generation of men and...,review_cluster_0
1,This movie makes people emotional and reminds ...,review_cluster_0
2,I could see the importance of occupation at life.,review_cluster_0
3,have always been a fan of stories about buildi...,review_cluster_0
4,It really liked playing with old people stereo...,review_cluster_0
...,...,...
642,"She has a child, and her husband is a stay at ...",review_cluster_19
643,"Continuing to defy clichés, the boss of this c...",review_cluster_19
644,She is a powerful business women that is 18 mo...,review_cluster_19
645,We soon learn that she is not the easiest pers...,review_cluster_19


In [75]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# 요약 파이프라인 초기화
model_name = 'facebook/bart-large-cnn'  # 예시로 BART 모델 사용
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 토크나이저를 사용하여 모델의 최대 입력 길이 확인
max_input_length = tokenizer.model_max_length
print(f"Model's maximum input length: {max_input_length}")

c:\python\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\python\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\서우\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode

Model's maximum input length: 1000000000000000019884624838656


In [16]:
#클러스터끼리 합치기

grouped = cluster_df.groupby('cluster')
cluster_texts = {}

for cluster, group in grouped:
    combined_text = ' '.join(group.dropna(subset=['review'])['review'])
    cluster_texts[f'{cluster}'] = combined_text

cluster_texts

{'review_cluster_0': 'A lot of shaming the new generation of men and glorifying the boomers. This movie makes people emotional and reminds us of why senior people are still very important in our society. I could see the importance of occupation at life. have always been a fan of stories about building friendships between young and old people, particularly in their exchange of wisdom which involves respect, mutual admiration, and compassion. It really liked playing with old people stereotypes but not in a bad way just in a ham-fisted way. Apparently this place only employs people under 35 who also happen to be loveable characters with some simple problems that desirous character can solve. But it is nice to see a movie valuing the wisdom that often comes with age, even if they overdo it quite a bit. Great work and lessons learned for life as a senior. Here\'s some feminism, take a kick in the nuts ageism, gender roles, boom! De\'Niro taught the milleniels how to be classy Appearing in g

In [79]:
###tept
# 요약 생성 함수: 입력 텍스트를 분할하여 요약
def generate_summary(text, max_length=1024):
    # 텍스트를 최대 길이에 맞게 분할
    words = text.split()
    summaries = []
    for i in range(0, len(words), max_length):
        chunk = ' '.join(words[i:i + max_length])
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=max_length)
        outputs = model.generate(inputs['input_ids'], max_length=130, min_length=20, length_penalty=2.0, num_beams=4)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        summaries.append(summary)
    return ' '.join(summaries)

# 데이터프레임에서 클러스터별 텍스트 추출
grouped = cluster_df.groupby('cluster')
cluster_texts = {}

for cluster, group in grouped:
    combined_text = ' '.join(group.dropna(subset=['review'])['review'])
    cluster_texts[f'{cluster}'] = combined_text

# 각 클러스터의 텍스트에 대해 요약 생성
summarized_clusters = {}
for cluster, text in cluster_texts.items():
    summary = generate_summary(text)
    summarized_clusters[f'review_cluster_{cluster}'] = summary

# 결과 확인
for cluster_name, summary in summarized_clusters.items():
    print(f"{cluster_name}: {summary}\n")


review_cluster_review_cluster_0: A lot of shaming the new generation of men and glorifying the boomers. This movie makes people emotional and reminds us of why senior people are still very important in our society. Heartwarming and lighthearted comedy that advocates respect for the seniors, family values and the courage to live your dreams. Cleverly touches on: bridging generation gap, gender roles, optimism and old age. High Pension explored the relationship between two generations and made you realize that we can all learn something from others. A casually emotional journey through everyday, though affluent, lives. Robert De Niro plays a wise, humble, dapper and old fashioned gent as the ever congenial retiree intern. It beautifully portrays how a Baby Boomer generation old school veteran brings peace and stability to a Millennial generation woman entrepreneur lost in the crazy business world. "The Intern" is about an older man who joins a company in their program for senior interns.

## 중간요약:

시니어와 관련되어 보이는 클러스터: 0번, 3번, 6번


In [38]:
cluster_df

,review,cluster
0,A lot of shaming the new generation of men and...,review_cluster_0
1,This movie makes people emotional and reminds ...,review_cluster_0
2,I could see the importance of occupation at life.,review_cluster_0
3,have always been a fan of stories about buildi...,review_cluster_0
4,It really liked playing with old people stereo...,review_cluster_0
...,...,...
642,"She has a child, and her husband is a stay at ...",review_cluster_19
643,"Continuing to defy clichés, the boss of this c...",review_cluster_19
644,She is a powerful business women that is 18 mo...,review_cluster_19
645,We soon learn that she is not the easiest pers...,review_cluster_19


In [95]:
# 관련 있는 것끼리 합치기

selected_clusters = ['review_cluster_0','review_cluster_3','review_cluster_6']

df_temp = cluster_df[cluster_df['cluster'].isin(selected_clusters)]

print(df_temp)

                                                review           cluster
0    A lot of shaming the new generation of men and...  review_cluster_0
1    This movie makes people emotional and reminds ...  review_cluster_0
2    I could see the importance of occupation at life.  review_cluster_0
3    have always been a fan of stories about buildi...  review_cluster_0
4    It really liked playing with old people stereo...  review_cluster_0
..                                                 ...               ...
326  Working completely online is initially a chall...  review_cluster_6
327  On top of the technology struggles, Ben is pla...  review_cluster_6
328  From the beginning, Ben is warned this task is...  review_cluster_6
329  Ben Whittaker, a seventy-year-old man who is s...  review_cluster_6
330  Nobody takes the time to get to know her accep...  review_cluster_6

[1214 rows x 2 columns]


## **3.텍스트 전처리**

In [90]:
def preprocessing_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  #숫자 제거
    text = re.sub(r'[^\w\s]', '', text)  #불필요한 문자 제거
    text = re.sub('[#]+[0-9a-zA-Z_]+', ' ', text) #해시태그 제거
    text = re.sub('[&]+[a-z]+', ' ', text) # &로 연결된거 제거
    text = re.sub('[^0-9a-zA-Z]', ' ', text) # 영어 아닌거 제거
    return text

In [96]:
df_temp['review'] = df_temp['review'].apply(preprocessing_text)

C:\Users\서우\AppData\Local\Temp\ipykernel_21388\750351828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['review'] = df_temp['review'].apply(preprocessing_text)


In [119]:
df_temp
df_temp_1 = df_temp.copy()

In [120]:
import nltk
from pattern.en import singularize
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

# NLTK 리소스 다운로드
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# 텍스트 예시
#txt = "Ben taught millenials to how to live"


#레마타이제이션
def lemmatization_text(text):
    # 토큰화 및 품사 태깅
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)

    # 표제어 추출을 위한 레마타이저 초기화
    lemmatizer = WordNetLemmatizer()

    # 결과를 저장할 리스트
    result = []

    # 각 토큰에 대해 표제어 추출
    for token, tag in tags:
        if tag.startswith('N'):  # 명사
            lemma = lemmatizer.lemmatize(token, pos='n')
            lemma = singularize(lemma)  # 복수형을 단수형으로 변환
        elif tag.startswith('V'):  # 동사
            lemma = lemmatizer.lemmatize(token, pos='v')
        elif tag.startswith('J'):  # 형용사
            lemma = lemmatizer.lemmatize(token, pos='a')
        elif tag.startswith('R'):  # 부사
            lemma = lemmatizer.lemmatize(token, pos='r')
        else:
            lemma = lemmatizer.lemmatize(token)  # 기본적으로 단어 변환
        result.append(lemma)
    
    return ' '.join(result)

# 결과 출력
df_temp_1['lemmatized_review'] = df_temp_1['review'].apply(lemmatization_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\서우\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\서우\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\서우\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [121]:
## 토큰화, 불용어 제거
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

# NLTK 리소스 다운로드
nltk.download('stopwords')
nltk.download('wordnet')

# 불용어 목록 로드
stop_words = set(stopwords.words('english'))
additional_stopwords = ['are', 're', 'is', 'be', 'a', 'the', 'an', 'could']
stop_words.update(additional_stopwords)

def process_review(text):
    tokens = word_tokenize(text)  # 텍스트를 토큰화
    tokens = [token for token in tokens if token.lower() not in stop_words]  # 불용어 제거
    return ' '.join(tokens)  # 토큰들을 문자열로 결합하여 반환

# 데이터프레임에 불용어 제거 적용
df_temp_1['processed_review'] = df_temp_1['lemmatized_review'].apply(process_review)

# 결과 확인
print(df_temp_1['processed_review'])

0             lot sham new generation man glorify boomer
1      movie make person emotional remind u senior pe...
2                         see importance occupation life
3      always fan story build friendship young old pe...
4      really like play old person stereotype bad way...
                             ...                        
326    work completely online initially challenge ben...
327    top technology struggle ben place work directl...
328    beginning ben warn task go tough one despite d...
329    ben whittaker seventyyearold man struggle find...
330                 nobody take time get know accept ben
Name: processed_review, Length: 1214, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\서우\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\서우\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [122]:
replacement_dict = {
    'youngster': 'young',
    'generational': 'generation',
    'elderly': 'old',
    'younger': 'young',
    'elder': 'old',
    'grandpa': 'old'
}

def apply_replacements(text):
    for old, new in replacement_dict.items():
        text = text.replace(old, new)
    return text

In [123]:
df_temp_1['processed_review'] = df_temp_1['processed_review'].apply(apply_replacements)

In [128]:
import pandas as pd
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# 모든 리뷰를 하나의 문자열로 결합
all_reviews = ' '.join(df_temp_1['processed_review'])

# 전체 리뷰에서 고유 단어 추출
unique_words = set(all_reviews.split())

# 결과 확인
sorted_unique_words = sorted(unique_words)

replacement_dict_final = {word: stemmer.stem(word) for word in set(sorted_unique_words)}

def stem_text(text):
    words = text.split()
    stemmed_words = [replacement_dict_final[word] for word in words if word in replacement_dict_final]
    return ' '.join(stemmed_words)

df_temp_1['processed_review_stem'] = df_temp_1['processed_review'].apply(stem_text)
df_temp_2 

## **4. 동시출현 행렬 생성하기**

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# 전체 문서에서 단어의 출현 빈도 계산
all_words = ' '.join(df_temp_1['processed_review']).split()
word_counts = Counter(all_words)

# 출현 빈도가 10번 이하인 단어 필터링
filtered_words = [word for word, count in word_counts.items() if count > 10]

# 중복 제거: 각 문서에서 단어 중복 제거
def remove_duplicates(text):
    words = text.split()
    return ' '.join(set(words))  # set을 사용해 중복 제거

# 문서 처리
df_temp_1['processed_review_unique'] = df_temp_1['processed_review'].apply(remove_duplicates)

In [138]:
# CountVectorizer 초기화
vectorizer = CountVectorizer(vocabulary=filtered_words)
X = vectorizer.fit_transform(df_temp_1['processed_review_unique'])

# 동시출현 행렬 생성
cooccurrence_matrix = X.T.dot(X)

# 단어 인덱스를 역맵핑하여 매핑
words = vectorizer.get_feature_names_out()
cooccurrence_df = pd.DataFrame(cooccurrence_matrix.toarray(), index=words, columns=words)

# 대각선 값(같은 단어의 동시출현)을 0으로 설정
for i in range(len(words)):
    cooccurrence_df.iat[i, i] = 0

# 결과 출력
print(cooccurrence_df)

# 엑셀로 저장
cooccurrence_df.to_excel('cooccurrence_matrix.xlsx', index=True)

            lot  new  generation  man  movie  make  person  remind  u  senior  \
lot           0    1           7    1      2     1       6       0  0       1   
new           1    0          10    5      5     4       6       0  0       2   
generation    7   10           0    4      7     3      17       4  0       7   
man           1    5           4    0      9     7       8       1  0       1   
movie         2    5           7    9      0     5       4       2  0       6   
...         ...  ...         ...  ...    ...   ...     ...     ... ..     ...   
coworker      1    3           0    2      0     1       1       0  0       0   
driver        1    0           1    0      0     0       0       0  0       0   
matt          0    0           0    0      0     0       0       0  0       0   
jule          1    2           1    4      4     1       1       0  0       4   
whittaker     1    1           0    1      4     1       0       0  0       2   

            ...  suit  husb